In [ ]:
import cv2 as cv
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import ipywidgets as widgets

import numpy as np

# Load image, resize and convert

In [ ]:
img = cv.imread('./walls.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
(img_height, img_width, _) = img.shape
img_ratio = 1000 / img_width
img = cv.resize(img, (1000, int(img_height * img_ratio)), interpolation=cv.INTER_AREA)
print('h: {}, w: {}'.format(int(img_height * img_ratio), 1000))

In [ ]:
plt.imshow(img)

Convert from rgb to hsv, it's easier to mask it that way

In [ ]:
hsv_img = cv.cvtColor(img, cv.COLOR_RGB2HSV)

Plot histograms of color spaces, both rgb and hsv

In [ ]:
for i, col in enumerate(('r', 'g', 'b')):
    hist_color_rgb = cv.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(hist_color_rgb, color=col)

In [ ]:
for i, col in enumerate(('r', 'g', 'b')):
    hist_color_hsv = cv.calcHist([hsv_img], [i], None, [256], [0, 256])
    plt.plot(hist_color_hsv, color=col)

# Color tweaking

Find parameters for extracting the mask (high and low colors for range limits) using sliders and looking for the best match.

In [ ]:
sliders_low = [widgets.IntSlider(value=0, min=0, max=180 if desc == 'h' else 255, step=1, description=desc) for desc in ('h', 's', 'v')]
sliders_high = [widgets.IntSlider(value=0, min=0, max=180 if desc == 'h' else 255, step=1, description=desc) for desc in ('h', 's', 'v')]
vbox = widgets.VBox([
    widgets.HBox(sliders_low),
    widgets.HBox(sliders_high),
])
vbox

Masking function. It diretly modifies the parameter. There is no need to use the return value

In [ ]:
def masker(mask):
    kernel = np.ones((5,5),np.uint8)
    mask2 = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask3 = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernel)
    masks = np.hstack((mask, mask2, mask3))
    return masks

Function that is called when slider value changes

In [ ]:
def on_hsv_change(_):
    _mask = cv.inRange(
        hsv_img,
        (int(sliders_low[0].value), int(sliders_low[1].value), int(sliders_low[2].value)),
        (int(sliders_high[0].value), int(sliders_high[1].value), int(sliders_high[2].value)),
    )
    masked = masker(_mask)
    plt.imshow(_mask)

[slider.observe(on_hsv_change) for slider in sliders_low + sliders_high]

# Create the mask

In [ ]:
WHITE_LOWER = (100, 0, 200)
WHITE_UPPER = (150, 50, 255)

BLUE_LOWER = (100, 190, 150)
BLUE_UPPER = (110, 255, 220)

_mask = cv.inRange(
    hsv_img,
    BLUE_LOWER,
    BLUE_UPPER
)

In [ ]:
masked = masker(_mask)

In [ ]:
plt.imshow(_mask)

# ROI corner finder

Find maze corners in specific regions of interest

In [ ]:
AREA_TOP = (0, 100)
AREA_BOTTOM = (350, 450) # based on ratio
AREA_LEFT = (250, 350)
AREA_RIGHT = (675, 775)

ORIGIN_TOP_LEFT = (0, 0)
ORIGIN_TOP_RIGHT = (0, 199)
ORIGIN_BOTTOM_LEFT = (199, 0)
ORIGIN_BOTTOM_RIGHT = (199, 199)

def roi_corner(mask, area_x, area_y, origin):
    roi_mask = mask[area_y[0]:area_y[1], area_x[0]:area_x[1]]
    roi_mask = np.float32(roi_mask)
    roi_corners = cv.cornerHarris(roi_mask, blockSize=3, ksize=3, k=0.05)
    
    roi_points = np.argwhere(roi_corners > 0.025 * roi_corners.max())

    roi_distances = np.array([np.linalg.norm(origin - p) for p in roi_points])
    roi_index = np.argmin(roi_distances)
    roi_point = roi_points[roi_index]

    return (roi_point[1] + area_x[0], roi_point[0] + area_y[0])

Plot corners to a new image

In [ ]:
img_dbg = img.copy()

In [ ]:
top_left_corner = roi_corner(_mask, AREA_LEFT, AREA_TOP, ORIGIN_TOP_LEFT)
print(top_left_corner)
img_dbg[top_left_corner[1]-10:top_left_corner[1]+10, top_left_corner[0]-10:top_left_corner[0]+10] = [255, 0, 0]

In [ ]:
top_right_corner = roi_corner(_mask, AREA_RIGHT, AREA_TOP, ORIGIN_TOP_RIGHT)
img_dbg[top_right_corner[1]-10:top_right_corner[1]+10, top_right_corner[0]-10:top_right_corner[0]+10] = [0, 255, 0]

In [ ]:
bottom_left_corner = roi_corner(_mask, AREA_LEFT, AREA_BOTTOM, ORIGIN_BOTTOM_LEFT)
img_dbg[bottom_left_corner[1]-10:bottom_left_corner[1]+10, bottom_left_corner[0]-10:bottom_left_corner[0]+10] = [0, 0, 255]

In [ ]:
bottom_right_corner = roi_corner(_mask, AREA_RIGHT, AREA_BOTTOM, ORIGIN_BOTTOM_RIGHT)
img_dbg[bottom_right_corner[1]-10:bottom_right_corner[1]+10, bottom_right_corner[0]-10:bottom_right_corner[0]+10] = [0, 255, 255]

In [ ]:
plt.imshow(img_dbg)

# Fix perspective

In [ ]:
old_perspective_points = np.float32([
    np.array(top_left_corner),
    np.array(bottom_left_corner),
    np.array(top_right_corner),
    np.array(bottom_right_corner)
])
new_perspective_points = np.float32([[0, 0], [0, 300], [300, 0], [300, 300]])
print(old_perspective_points)
print(new_perspective_points)

In [ ]:
perspective_matrix = cv.getPerspectiveTransform(old_perspective_points, new_perspective_points)

In [ ]:
height, width = img.shape[:2]
fixed_perspective_img = cv.warpPerspective(img, perspective_matrix, (300, 300))

In [ ]:
plt.imshow(fixed_perspective_img)